# Experimento TCC

## Libs

In [1]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from psutil import cpu_count
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO, Gradient
from modules.utils import folds_run, sklearn_folds_run

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
dtypes = {
    'tau1': float, 'tau2': float, 'tau3': float, 'tau4': float,
    'p1': float, 'p2': float,'p3': float, 'p4': float, 
    'g1': float, 'g2': float, 'g3': float, 'g4': float}

EletricalGrid = pd.read_csv('dataset/Data_for_UCI_named.csv',
                            dtype=dtypes, nrows=300).drop('stab', axis=1)

In [5]:
X, y = EletricalGrid.drop('stabf',axis=1), EletricalGrid.stabf 

In [6]:
K_neighbors = 5
max_iter= 10

In [7]:
my_pso_options = {
    'optimization_func':PSO, 'max_iter':max_iter, 'k':K_neighbors, 
    'cpu_count':cpu_count(),'swarm_size':100
}

sklearn_options = {
    'X': np.asarray(X), 'y': np.asarray(y),
    'k': K_neighbors, 'model_class': NeighborhoodComponentsAnalysis,
    'model_options': {'n_components':None, 'max_iter':max_iter,'random_state':42}
}

In [8]:
nca_pso_results = [folds_run(X, y, model_class=NCA, model_options=my_pso_options) for _ in tqdm(range(10))]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [05:15<00:00, 31.58s/it]


In [9]:
nca_gradient_based_results = [sklearn_folds_run(**sklearn_options) for _ in tqdm(range(10))]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.27s/it]
